In [1]:
import argparse
import base64
import csv
import datetime
import difflib
import io
import json

import logging
import math
import operator
import os
import pickle
from collections import defaultdict
from os.path import expanduser
from pathlib import Path
from sentence_transformers import SentenceTransformer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from dask.distributed import Client
# import dask.dataframe as dd
import pandas as pd


import nltk
import dash
import sacrebleu
import dash_bootstrap_components as dbc
import diff_match_patch
import editdistance
import jiwer
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import tqdm
import spacy
from indicnlp.tokenize import sentence_tokenize
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from plotly import express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize


import timeit
import json
from collections import Counter
import dask
from dask.distributed import Client
import gc  # for garbage collection

In [2]:
data_filename = "filepath.json"

In [3]:
with open(data_filename, 'r', encoding='utf8') as f:
        json_content = f.read()

In [4]:
parsed_data = json.loads(json_content)

In [5]:
parsed_data

{'path1': '/Users/crysiswar999/Documents/MTP/transliteration/hin/transliteration_hin.csv'}

### Accuracy

In [6]:
def calculate_character_accuracy(reference, prediction):
  num_correct = sum(1 for c1, c2 in zip(reference, prediction) if c1 == c2)
  accuracy = num_correct / len(reference) * 100
  return accuracy

# Example usage
reference = Hindi_token
prediction = Hindi_token

accuracy = calculate_character_accuracy(reference, prediction)
print(f"Character accuracy: {accuracy:.2f}%")


NameError: name 'Hindi_token' is not defined

### Levenstein Distance

In [ ]:
from nltk.metrics import edit_distance

reference = df.iloc(0)[0]['Hindi']
prediction = reference

distance = edit_distance(reference, prediction)

print(f"Levenshtein Distance: {distance}")


### Bleu Score

In [ ]:
from sacrebleu import BLEU

reference = df.iloc(0)[0]['Hindi']
prediction = reference

bleu = sentence_bleu(reference,prediction)
print(f"Bleu score: {bleu}")


### Rouge Score

In [ ]:
import evaluate
rouge = evaluate.load('rouge')


In [ ]:
reference = df.iloc(0)[0]['Hindi']
prediction = reference
results = rouge.compute(predictions=prediction,references=reference)
print(f"rouge score: {results}")

### Character Error Rate

In [ ]:
from datasets import load_metric

metric = load_metric("cer")

references = df.iloc(0)[0]['Hindi']
predictions = references

results = metric.compute(references=references, predictions=predictions)

print(f"character error rate: {results}")


### Word Error Rate (if data is given in sentences)


### Load_data funtion (Sequential)

In [24]:
def load_data(data_filename,comparison_mode,names_compared,total):
    word_count = total
    char_distribution = {}
    vocab = {}
    avg_length = {}
    for x in names_compared:
        char_distribution[x] = {}
    df = pd.read_csv(data_filename,names = names_compared)
    print(df.head())
    for x in names_compared:
        length = 0
        for y in range(len(df)):
            words = df.iloc(0)[y][x]
            length += len(words)
            for ch in words:
                if(ch in char_distribution[x].keys()):
                    char_distribution[x][ch] += 1
                else:
                    char_distribution[x][ch] = 1
        avg_length[x] = length/len(df[x])
    for x in names_compared:
        vocab[x] = sorted(char_distribution[x].keys())
    word_count = len(df)

    return df,char_distribution,word_count,vocab,avg_length


In [5]:
data,char_distribution,word_count,vocab,avg_length = load_data("/Users/crysiswar999/Documents/MTP/transliteration/hin/transliteration_hin.csv",True,['English','Hindi','transliterated_hindi'],10000)

       English       Hindi transliterated_hindi
0  shastragaar  शस्त्रागार           शस्त्रागार
1      bindhya    बिन्द्या             बिन्द्या
2    kirankant    किरणकांत             किरणकांत
3  yagyopaveet   यज्ञोपवीत            यज्ञोपवीत
4      ratania     रटानिया              रटानिया


In [6]:
vocab.keys()

dict_keys(['English', 'Hindi', 'transliterated_hindi'])

In [7]:
avg_length

{'English': 8.7069921875,
 'Hindi': 7.79576171875,
 'transliterated_hindi': 7.79576171875}

In [6]:
app = dash.Dash(
        __name__,
        suppress_callback_exceptions=True,
        external_stylesheets=[dbc.themes.BOOTSTRAP,'https://codepen.io/chriddyp/pen/bWLwgP.css'],
        title="Text Data Explorer",
    )

In [48]:
names_compared = ['English','Hindi','transliterated_hindi']

In [26]:
for key in parsed_data:
    print(names_compared[3:5])

['English', 'Odia']


In [49]:
count = 0
data = {}
for key in parsed_data:
    translation_pair = '-'.join(names_compared[count:count + 2])
    print(translation_pair)
    temp = load_data(parsed_data[key],True,names_compared[count:count+3],10000)
    data[translation_pair] = temp
    count += 3

    

English-Hindi
       English       Hindi transliterated_hindi
0  shastragaar  शस्त्रागार           शस्त्रागार
1      bindhya    बिन्द्या             बिन्द्या
2    kirankant    किरणकांत             किरणकांत
3  yagyopaveet   यज्ञोपवीत            यज्ञोपवीत
4      ratania     रटानिया              रटानिया


In [50]:
def plot_histogram(data,xlabel,ylabel,count):
    fig = px.histogram(
        x=list(data.keys())[:count],  # Use the characters as x values
        y=list(data.values())[:count],  # Use the frequencies as y values
        nbins=50,
        log_y=True,
        labels={'x': xlabel, 'y': ylabel},  # Labels for axes
        opacity=0.5,
        color_discrete_sequence=['cornflowerblue'],
        height=200,
    )
    fig.update_layout(showlegend=False, margin=dict(l=0, r=0, t=0, b=0, pad=0))
    return fig

In [84]:
figures_hist = {}
count = 0
for x in data:
    char_distribution = {}
    char_distribution[names_compared[count]] = dict(sorted(data[x][1][names_compared[count]].items(),key = lambda x : x[1],reverse = True))
    char_distribution[names_compared[count+1]] = dict(sorted(data[x][1][names_compared[count + 1]].items(),key = lambda x : x[1],reverse = True))
    char_distribution[names_compared[count+2]] = dict(sorted(data[x][1][names_compared[count + 2]].items(),key = lambda x : x[1],reverse = True))
    figures_hist[x + str(count)] = ['Character Distribution ' + names_compared[count], plot_histogram(char_distribution[names_compared[count]],'characters','Frequency',20)]
    figures_hist[x + str(count + 1)] = ['Character_Distribution ' + names_compared[count + 1],plot_histogram(char_distribution[names_compared[count + 1]],'characters','Frequency',20)] 
    figures_hist[x + str(count + 2)] = ['Character_Distribution ' + names_compared[count + 2],plot_histogram(char_distribution[names_compared[count + 2]],'characters','Frequency',20)] 
    
    

    

In [83]:
dict(sorted(data['English-Hindi'][1][names_compared[count]].items(),key = lambda x : x[1],reverse=True))

{'a': 89517,
 'i': 33360,
 'n': 31315,
 'r': 29858,
 'h': 29277,
 'e': 24016,
 't': 22158,
 's': 21925,
 'o': 17640,
 'l': 16068,
 'd': 15918,
 'u': 15085,
 'k': 14792,
 'm': 14146,
 'p': 11099,
 'y': 10076,
 'c': 9753,
 'g': 9646,
 'b': 9532,
 'v': 7966,
 'j': 5168,
 'w': 3263,
 'f': 2414,
 'z': 1112,
 'x': 421,
 'q': 273}

In [52]:
figures_hist.keys()

dict_keys(['English-Hindi0', 'English-Hindi1', 'English-Hindi2'])

In [53]:
comparison_mode = True

In [54]:
if comparison_mode == True:
    tab_labels = []
    for x in data:
        tab_labels += [
                    dbc.Tab(label = x,tab_id  = x)
                ]

In [55]:
data

{'English-Hindi': (           English       Hindi transliterated_hindi
  0      shastragaar  शस्त्रागार           शस्त्रागार
  1          bindhya    बिन्द्या             बिन्द्या
  2        kirankant    किरणकांत             किरणकांत
  3      yagyopaveet   यज्ञोपवीत            यज्ञोपवीत
  4          ratania     रटानिया              रटानिया
  ...            ...         ...                  ...
  51195        toned        टोंड                 टोंड
  51196   mutanaazaa    मुतनाज़ा             मुतनाज़ा
  51197    asahmaton     असहमतों              असहमतों
  51198    sulgaayin    सुलगायीं             सुलगायीं
  51199  anchuthengu   अंचुतेंगु            अंचुतेंगु
  
  [51200 rows x 3 columns],
  {'English': {'s': 21925,
    'h': 29277,
    'a': 89517,
    't': 22158,
    'r': 29858,
    'g': 9646,
    'b': 9532,
    'i': 33360,
    'n': 31315,
    'd': 15918,
    'y': 10076,
    'k': 14792,
    'o': 17640,
    'p': 11099,
    'v': 7966,
    'e': 24016,
    'c': 9753,
    'm': 14146,
    'u': 

In [56]:
if comparison_mode == True:
    tab_content = {}
    for x in range(0,len(names_compared)-1,3):
        f_name1 = names_compared[x]
        f_name2 = names_compared[x+1]
        f_name3 = names_compared[x+2]
        translation_pair = '-'.join([f_name1,f_name2])
        sub_tab_labels = []
        sub_tab_labels += [dbc.Tab(label = f_name1,tab_id = f_name1)]
        sub_tab_labels += [dbc.Tab(label = f_name2,tab_id = f_name2)]
        sub_tab_labels += [dbc.Tab(label = f_name3,tab_id = f_name3)]
        sub_tab_content = {}
        sub_tab_content[f_name1] = dbc.Card(
            dbc.CardBody(
                [
            dbc.Row(dbc.Col(html.H6(children = f_name1),className = 'text-secondary'),className = 'mt-3'),
            dbc.Row(
                [
                    dbc.Col(html.Div('Word Count', className='text-secondary'), width=3, className='border-end'),
                    dbc.Col(html.Div('Vocabulary', className='text-secondary'), width=3, className='border-end'),
                ],
                className='bg-light mt-2 rounded-top border-top border-start border-end',
            ),
            dbc.Row(
                [
                    dbc.Col(
                        html.H5(
                            data[translation_pair][2],
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                    dbc.Col(
                        html.H5(
                            '{} characters'.format(len(data[translation_pair][1][f_name1])),
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                ],
                className='bg-light rounded-bottom border-bottom border-start border-end',
            ),
            dbc.Row(dbc.Col(html.H5(children='Alphabet'), class_name='text-secondary'), class_name='mt-3'),
        dbc.Row(
            dbc.Col(html.Div('{}'.format(sorted(data[translation_pair][3][f_name1]))),), class_name='mt-2 bg-light font-monospace rounded border'
        ),
            dbc.Row(dbc.Col(html.H5(figures_hist[translation_pair + str(x)][0]), class_name='text-secondary'), class_name='mt-3'),
            dbc.Row(dbc.Col(dcc.Graph(id='char_dist_graph' + str(x), figure=figures_hist[translation_pair + str(x)][1])),),
            dbc.Row(dbc.Col(html.H6('Top K characters Slider'),class_name = 'text-secondary'),class_name = 'mt-3'),
            dbc.Row(html.Div([dcc.Slider(
                1,
                20,
                step=None,
                value=10,
                marks={str(num): str(num) for num in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]},
                id='char_slider' + str(x),
                className = 'slider'
            )],style = {}
                )),
            
        ]
            )
        )
        sub_tab_content[f_name2] = dbc.Card(
            dbc.CardBody(
                [
            dbc.Row(dbc.Col(html.H6(children = f_name2),className = 'text-secondary'),className = 'mt-3'),
            dbc.Row(
                [
                    dbc.Col(html.Div('Word Count', className='text-secondary'), width=3, className='border-end'),
                    dbc.Col(html.Div('Vocabulary', className='text-secondary'), width=3, className='border-end'),
                ],
                className='bg-light mt-2 rounded-top border-top border-start border-end',
            ),
            dbc.Row(
                [
                    dbc.Col(
                        html.H5(
                            data[translation_pair][2],
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                    dbc.Col(
                        html.H5(
                            '{} characters'.format(len(data[translation_pair][1][f_name2])),
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                ],
                className='bg-light rounded-bottom border-bottom border-start border-end',
            ),
            dbc.Row(dbc.Col(html.H5(children='Alphabet'), class_name='text-secondary'), class_name='mt-3'),
        dbc.Row(
            dbc.Col(html.Div('{}'.format(sorted(data[translation_pair][3][f_name2]))),), class_name='mt-2 bg-light font-monospace rounded border'
        ),
            dbc.Row(dbc.Col(html.H5(figures_hist[translation_pair + str(x+1)][0]), class_name='text-secondary'), class_name='mt-3'),
            dbc.Row(dbc.Col(dcc.Graph(id='char_dist_graph' + str(x+1), figure=figures_hist[translation_pair + str(x+1)][1])),),
            dbc.Row(dbc.Col(html.H6('Top K characters Slider'),class_name = 'text-secondary'),class_name = 'mt-3'),
            dbc.Row(html.Div([dcc.Slider(
                1,
                20,
                step=None,
                value=10,
                marks={str(num): str(num) for num in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]},
                id='char_slider' + str(x),
                className = 'slider'
            )],style = {}
                )),
            
        ]
            )
        )
        sub_tab_content[f_name3] = dbc.Card(
            dbc.CardBody(
                [
            dbc.Row(dbc.Col(html.H6(children = f_name3),className = 'text-secondary'),className = 'mt-3'),
            dbc.Row(
                [
                    dbc.Col(html.Div('Word Count', className='text-secondary'), width=3, className='border-end'),
                    dbc.Col(html.Div('Vocabulary', className='text-secondary'), width=3, className='border-end'),
                ],
                className='bg-light mt-2 rounded-top border-top border-start border-end',
            ),
            dbc.Row(
                [
                    dbc.Col(
                        html.H5(
                            data[translation_pair][2],
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                    dbc.Col(
                        html.H5(
                            '{} characters'.format(len(data[translation_pair][1][f_name3])),
                            className='text-center p-1',
                            style={'color': 'cornflowerblue', 'opacity': 0.7},
                        ),
                        width=3,
                        className='border-end',
                    ),
                ],
                className='bg-light rounded-bottom border-bottom border-start border-end',
            ),
            dbc.Row(dbc.Col(html.H5(children='Alphabet'), class_name='text-secondary'), class_name='mt-3'),
        dbc.Row(
            dbc.Col(html.Div('{}'.format(sorted(data[translation_pair][3][f_name3]))),), class_name='mt-2 bg-light font-monospace rounded border'
        ),
            dbc.Row(dbc.Col(html.H5(figures_hist[translation_pair + str(x+2)][0]), class_name='text-secondary'), class_name='mt-3'),
            dbc.Row(dbc.Col(dcc.Graph(id='char_dist_graph' + str(x), figure=figures_hist[translation_pair + str(x + 2)][1])),),
            dbc.Row(dbc.Col(html.H6('Top K characters Slider'),class_name = 'text-secondary'),class_name = 'mt-3'),
            dbc.Row(html.Div([dcc.Slider(
                1,
                20,
                step=None,
                value=10,
                marks={str(num): str(num) for num in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]},
                id='char_slider' + str(x),
                className = 'slider'
            )],style = {}
                )),
            
        ]
            )
        )
        tab_content[translation_pair] = dbc.Card(
        dbc.CardBody(
            [
                dbc.Tabs(
                    [
                        dbc.Tab(sub_tab_content[f_name1], label=f_name1, tab_id=f_name1),
                        dbc.Tab(sub_tab_content[f_name2], label=f_name2, tab_id=f_name2),
                        dbc.Tab(sub_tab_content[f_name3], label=f_name3, tab_id=f_name3),
                    ],
                    active_tab=f_name1,
                )
            ]
        ),
        className="mt-3",
    )

        active = translation_pair



## Backend Samples

In [57]:
def calculate_character_accuracy(reference, prediction):
    num_correct = sum(1 for c1, c2 in zip(reference, prediction) if c1 == c2)
    accuracy = num_correct / len(reference) * 100
    return accuracy

# # Example usag,x[e
# reference = Hindi_token
# prediction = Hindi_token

# accuracy = calculate_character_accuracy(reference, prediction)
# print(f"Character accuracy: {accuracy:.2f}%")


In [58]:
from nltk.metrics import edit_distance

# reference = df.iloc(0)[0]['Hindi']
# prediction = reference

# distance = edit_distance(reference, prediction)

# print(f"Levenshtein Distance: {distance}")

In [59]:
from sacrebleu import BLEU

# reference = df.iloc(0)[0]['Hindi']
# prediction = reference

# bleu = sentence_bleu(reference,prediction)
# print(f"Bleu score: {bleu}")

In [60]:
import evaluate
rouge = evaluate.load('rouge')
# reference = df.iloc(0)[0]['Hindi']
# prediction = reference
# print(prediction,reference)
# results = rouge.compute(predictions=prediction,references=reference)
# print(f"rouge score: {results}")

In [61]:
def calculate_jaccard_score(reference,candidate):
    total = len(reference) + len(candidate)
    curr = 0
    ref_dict = {}
    can_dict = {}
    for x in reference:
        if(x in ref_dict.keys()):
            ref_dict[x] += 1
        else:
            ref_dict[x] = 1
    for x in candidate:
        if(x in can_dict.keys()):
            can_dict[x] += 1
        else:
            can_dict[x] = 1
    for x in ref_dict.keys():
        ref = ref_dict[x]
        can = 0
        if x in can_dict.keys():
            can = can_dict[x]
        curr += 2*min(ref,can)
    return curr/total

        


In [62]:
from datasets import load_metric

metric = evaluate.load("cer")

# references = df.iloc(0)[0]['Hindi']
# predictions = references

# results = metric.compute(references=references, predictions=predictions)

# print(f"character error rate: {results}")

In [63]:
samples_data = {}
for x in range(0,len(names_compared)-1,3):
        f_name1 = names_compared[x]
        f_name2 = names_compared[x+1]
        f_name3 = names_compared[x+2]
        translation_pair = '-'.join([f_name1,f_name2])
        samples_data[translation_pair] = {}
        samples_data[translation_pair]['Accuracy'] = []
        samples_data[translation_pair]['Levenshtein Distance'] = []
        samples_data[translation_pair]['Jaccard Similarity'] = []
        samples_data[translation_pair]['CER'] = []
        df = data[translation_pair][0]
        count = 0
        total_accuracy = 0
        total_lev = 0
        total_jaccard = 0
        total_cer = 0
        for x in df.iterrows():
                reference = x[1][f_name3]
                candidate = x[1][f_name2]
                count += 1
                a = calculate_character_accuracy(reference,candidate)
                b = edit_distance(reference,candidate)
                c = calculate_jaccard_score(reference,candidate)
                d = metric.compute(references = reference,predictions = candidate)
                total_accuracy += a
                total_lev += b
                total_jaccard += c
                total_cer += d
                # print(reference,candidate)
                samples_data[translation_pair]['Accuracy'].append(a)
                samples_data[translation_pair]['Levenshtein Distance'].append(b)
                samples_data[translation_pair]['Jaccard Similarity'].append(c)
                samples_data[translation_pair]['CER'].append(d)
print(samples_data['English-Hindi']['CER'])




[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

## Samples page


In [64]:
reference = True

In [65]:
samples_data['English-Hindi'].keys()

dict_keys(['Accuracy', 'Levenshtein Distance', 'Jaccard Similarity', 'CER'])

In [66]:
PAGE_SIZE = 10

In [67]:
if comparison_mode == True:
    if reference == True:
        sample_tab_content = {}
        for x in range(0,len(names_compared) - 1,3):
            f_name1 = names_compared[x]
            f_name2 = names_compared[x+1]
            f_name3 = names_compared[x+2]
            col = []
            col += [{'name':f_name1,'id':f_name1}]
            col += [{'name':f_name2,'id':f_name2}]
            col += [{'name':f_name3,'id':f_name3}]
            col += [{'name':'Accuracy','id':'Accuracy','type': 'numeric'}]
            col += [{'name':'Levenshtein Distance','id':'Levenshtein Distance','type':'numeric'}]
            col += [{'name':'Jaccard Similarity','id':'Jaccard Similarity','type':'numeric'}]
            col += [{'name':'CER','id':'CER','type':'numeric'}]

            translation_pair = '-'.join([f_name1,f_name2])
            d = pd.DataFrame.from_dict(samples_data[translation_pair])
            df = pd.merge(data[translation_pair][0],d,left_index=True,right_index=True)
            sample_tab_content[translation_pair] = dbc.Card(dbc.CardBody(html.Div([
                    html.Div(style = {'display':'none'},id = 'names' + str(x),children = [f_name1,f_name2]),
                    dash_table.DataTable(
                        id='table' + str(x),
                        columns=col,
                        data=df.to_dict('records'),
                        style_table={'width': '100%'},
                        row_selectable='single',  # Allow multiple row selection
                        selected_rows=[],
                        page_size = PAGE_SIZE,
                        page_current = 0,
                        # style_data_conditional=[
                        #     {
                        #         'if': {'row_index': 0},  # Apply the style to the first row
                        #         'display': 'none',       # Hide the first row
                        #     },
                        # ],
                        style_cell={'overflow': 'hidden', 'textOverflow': 'ellipsis', 'maxWidth': 0, 'textAlign': 'center'},
                                style_header={
                                    'color': 'text-primary',
                                    'text_align': 'center',
                                    'height': 'auto',
                                    'whiteSpace': 'normal',
                                },
                                css=[
                                    {'selector': '.dash-spreadsheet-menu', 'rule': 'position:absolute; bottom: 8px'},
                                    {'selector': '.dash-filter--case', 'rule': 'display: none'},
                                    {'selector': '.column-header--hide', 'rule': 'display: none'},
                                ],  # Initialize with no selected rows
                    ),
                
                ])),className = 'mt-3')
            



In [68]:
samples_layout = []
samples_layout += [dbc.Row(dbc.Col(html.H5(children='Sample Statistics'), className='text-secondary'), className='mt-3')]
samples_layout += [html.Div(
    [
        dbc.Tabs(
            tab_labels,
            id="sample_tabs",
            active_tab=names_compared[0] + '-' + names_compared[1],
        ),
        html.Div(id="samples-tab-content"),
    ]
)]
samples_layout = html.Div(samples_layout)
@app.callback(Output("samples-tab-content", "children"), [Input("sample_tabs", "active_tab")])
def switch_tab(at):
    return sample_tab_content[at]

In [69]:
stats_layout = []
stats_layout += [dbc.Row(dbc.Col(html.H5(children='Global Statistics'), className='text-secondary'), className='mt-3')]
stats_layout += [html.Div(
    [
        dbc.Tabs(
            tab_labels,
            id="tabs",
            active_tab=active,
        ),
        html.Div(id="tab-content"),
    ]
)]
stats_layout = html.Div(stats_layout)

In [70]:
@app.callback(Output("tab-content", "children"), [Input("tabs", "active_tab")])
def switch_tab(at):
    return tab_content[at]

In [71]:
# samples_layout = []
comparison_layout = []

In [72]:
app.layout = html.Div(
            [
                dcc.Location(id='url', refresh=False),
                dbc.NavbarSimple(
                    children=[
                        dbc.NavItem(dbc.NavLink('Statistics', id='stats_link', href='/', active=True)),
                        dbc.NavItem(dbc.NavLink('Samples', id='samples_link', href='/samples')),
                        dbc.NavItem(dbc.NavLink('Comparison tool', id='comp_tool', href='/comparison')),
                    ],
                    brand='Text Data Explorer',
                    sticky='top',
                    color='orange',
                    dark=True,
                ),
                dbc.Container(id='page-content'),
            ]
        )
@app.callback(
    [
        Output('page-content', 'children'),
        Output('stats_link', 'active'),
        Output('samples_link', 'active'),
        Output('comp_tool', 'active'),
    ],
    [Input('url', 'pathname')],
)
def nav_click(url):
    if url == '/samples':
        return [samples_layout, False, True, False]
    elif url == '/comparison':
        return [comparison_layout, False, False, True]
    else:
        return [stats_layout, True, False, False]


In [73]:
app.run(port = 8064,debug=True)